In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

  Activating project at `D:\Science\Code\Julia\envr`


In [88]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [89]:
function TM(u, p, t)

    U(y) = U0 + ΔU0 / ( 1 + exp( -50*(y-ythr) ) )
    σ(x) = 1 / ( 1 + exp( -20*(x-xthr) ) )

    E, x, y = u
    α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0 = p

    du1 = (-E + α * log( 1 + exp( (J * U(y) * x * E + I0  ) / (α) ) ) ) / τ
    du2 = (1-x)/τD - U(y)*x*E
    du3 = (-y)/τy+β*σ(x)
    return SVector(du1, du2, du3)
end;

In [90]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [92]:
t = 500.0; tt = 250.0; tstep = 0.001
time_attract = 100.0
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [93]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [94]:
u0 = SA[17.6055, 0.6181, 0.429232]

3-element SVector{3, Float64} with indices SOneTo(3):
 17.6055
  0.6181
  0.429232

In [95]:
I0range = range( -1.40, -1.709, length = 100 )
U0range = range( 0.3,  0.47, length = 100)

0.3:0.0017171717171717172:0.47

In [96]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [98]:
function init_ds(U0_, u0_lc)
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0range[1]]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    save("LS_100x100.jld", "data", Λs)
    save("u0s_100x100.jld", "data", u0s)
end

save_tofile (generic function with 1 method)

In [99]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    output(idx_U0, U0_, u0_lc) # Вывод
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ΛΛ = spectrum(ds) # Вычисление спектра
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    output_end_iter(ΛΛ, u0_lc) # Вывод
    save_tofile()
    separate()
    
end

Индекс U0: 1; U0: 0.3
Начальные условия: [17.6055, 0.6181, 0.429232]
Спектр: [0.0018452838036348827, -2.760050665692471, -2.784307433770536]
Последняя точка: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]

---------
Индекс U0: 2; U0: 0.3017171717171717
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [0.0017271176223297377, -2.7790897710281586, -2.7901901086580736]
Последняя точка: [13.679802545102358, 0.6050830537719935, 0.424075623774622]

---------
Индекс U0: 3; U0: 0.30343434343434345
Начальные условия: [13.679802545102358, 0.6050830537719935, 0.424075623774622]
Спектр: [0.002722984936765527, -2.7856868621665902, -2.8039446942709514]
Последняя точка: [4.135391084978783, 0.8034609500203724, 0.41919621615864416]

---------
Индекс U0: 4; U0: 0.3051515151515152
Начальные условия: [4.135391084978783, 0.8034609500203724, 0.41919621615864416]
Спектр: [0.01045484510156406, -2.801198601116878, -2.8131899025556826]
Последняя точка: [9.0408

Последняя точка: [15.64490424562619, 0.7261374090228995, 0.4207264589354413]

---------
Индекс U0: 31; U0: 0.3515151515151515
Начальные условия: [15.64490424562619, 0.7261374090228995, 0.4207264589354413]
Спектр: [0.009092929267742139, -1.8124882819489456, -4.2847587310093775]
Последняя точка: [19.744002802499978, 0.5824652226272542, 0.41565692538336435]

---------
Индекс U0: 32; U0: 0.35323232323232323
Начальные условия: [19.744002802499978, 0.5824652226272542, 0.41565692538336435]
Спектр: [0.0031618524580984643, -1.8325183745216107, -4.270233655672061]
Последняя точка: [16.962295813139043, 0.5698727205818499, 0.41275845701023584]

---------
Индекс U0: 33; U0: 0.35494949494949496
Начальные условия: [16.962295813139043, 0.5698727205818499, 0.41275845701023584]
Спектр: [0.007652719389689789, -1.8660086112301757, -4.250191824120175]
Последняя точка: [19.332881119829132, 0.5765623596506378, 0.414275407914229]

---------
Индекс U0: 34; U0: 0.3566666666666667
Начальные условия: [19.33288111

Начальные условия: [18.541816767750635, 0.5688323998830086, 0.40150982888752595]
Спектр: [-0.0017655329129357483, -3.1119081684360603, -3.1347869023648114]
Последняя точка: [3.931412123175301, 0.8190739722816605, 0.3964523233822517]

---------
Индекс U0: 61; U0: 0.403030303030303
Начальные условия: [3.931412123175301, 0.8190739722816605, 0.3964523233822517]
Спектр: [0.012482233625764843, -3.1242022040086006, -3.1296600627890494]
Последняя точка: [4.00678786068436, 0.8210689408218941, 0.3965418756724706]

---------
Индекс U0: 62; U0: 0.40474747474747474
Начальные условия: [4.00678786068436, 0.8210689408218941, 0.3965418756724706]
Спектр: [0.007830050788140945, -3.1194689600042222, -3.130286487545363]
Последняя точка: [17.901708415023176, 0.5688685688755021, 0.39956476209401276]

---------
Индекс U0: 63; U0: 0.4064646464646465
Начальные условия: [17.901708415023176, 0.5688685688755021, 0.39956476209401276]
Спектр: [0.0053927353159709856, -3.1158645479440454, -3.1302306037087972]
Последня

Последняя точка: [12.320556900781929, 0.7417570388798931, 0.3869989053820769]

---------
Индекс U0: 90; U0: 0.4528282828282828
Начальные условия: [12.320556900781929, 0.7417570388798931, 0.3869989053820769]
Спектр: [0.0035638438899757738, -3.0220133871437986, -3.0377615190118585]
Последняя точка: [5.920668971569371, 0.7994788365624969, 0.38067185504517626]

---------
Индекс U0: 91; U0: 0.45454545454545453
Начальные условия: [5.920668971569371, 0.7994788365624969, 0.38067185504517626]
Спектр: [0.008950142588558915, -3.018540044735569, -3.039278969651705]
Последняя точка: [5.815275280138603, 0.7970972119553386, 0.37908932117158045]

---------
Индекс U0: 92; U0: 0.45626262626262626
Начальные условия: [5.815275280138603, 0.7970972119553386, 0.37908932117158045]
Спектр: [0.006363554357867754, -2.2017331038134382, -3.854572561365201]
Последняя точка: [6.521355571170218, 0.793564152284932, 0.37953478418297687]

---------
Индекс U0: 93; U0: 0.457979797979798
Начальные условия: [6.5213555711702

In [ ]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1
            continue
        end
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ΛΛ = spectrum(ds) # Вычисление спектра
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        output_end_iter(ΛΛ, u0_lc)
        save_tofile()
        separate()
    end
end

Индекс I0: 2; I0: -1.4031212121212122
Индекс U0: 1; U0: 0.3
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [-0.000670836877794711, -2.7531631238801424, -2.7606284501940617]
Последняя точка: [17.947682119259355, 0.628448047402727, 0.4302185414905105]

---------
Индекс I0: 3; I0: -1.4062424242424243
Индекс U0: 1; U0: 0.3
Начальные условия: [17.947682119259355, 0.628448047402727, 0.4302185414905105]
Спектр: [0.005570838420016841, -2.734315174175925, -2.753492002798311]
Последняя точка: [10.013059485051526, 0.6381962735641662, 0.42086435071475203]

---------
Индекс I0: 4; I0: -1.4093636363636364
Индекс U0: 1; U0: 0.3
Начальные условия: [10.013059485051526, 0.6381962735641662, 0.42086435071475203]
Спектр: [0.006461385593928904, -2.721259569253147, -2.7328112027866833]
Последняя точка: [4.16094914640017, 0.8031422006788425, 0.4204654104802386]

---------
Индекс I0: 5; I0: -1.4124848484848485
Индекс U0: 1; U0: 0.3
Начальные условия: [4.16094914640017, 

Спектр: [0.008508605032155085, -0.1739097318565578, -4.455927603618406]
Последняя точка: [4.642271394583186, 0.8305972230144185, 0.4303908906186051]

---------
Индекс I0: 30; I0: -1.4905151515151516
Индекс U0: 1; U0: 0.3
Начальные условия: [4.642271394583186, 0.8305972230144185, 0.4303908906186051]
Спектр: [0.008140753391514625, -0.11042446304246804, -4.482007935875563]
Последняя точка: [4.077542981154247, 0.7973691276308977, 0.4224075293422264]

---------
Индекс I0: 31; I0: -1.4936363636363637
Индекс U0: 1; U0: 0.3
Начальные условия: [4.077542981154247, 0.7973691276308977, 0.4224075293422264]
Спектр: [0.011458982246037471, -0.04657993091832769, -4.514619203925113]
Последняя точка: [4.372743426164828, 0.7794762839482353, 0.4209588122552633]

---------
Индекс I0: 32; I0: -1.4967575757575757
Индекс U0: 1; U0: 0.3
Начальные условия: [4.372743426164828, 0.7794762839482353, 0.4209588122552633]
Спектр: [0.008354500065976293, -0.006590084039196809, -4.516704167095286]
Последняя точка: [3.7992


---------
Индекс I0: 57; I0: -1.5747878787878788
Индекс U0: 1; U0: 0.3
Начальные условия: [2.6625029987830877, 0.8660669799333698, 0.4335555964350808]
Спектр: [0.004540322516539283, -0.4329236779617321, -3.57776999029341]
Последняя точка: [9.407038332584772, 0.7471403966175919, 0.43219718045477806]

---------
Индекс I0: 58; I0: -1.577909090909091
Индекс U0: 1; U0: 0.3
Начальные условия: [9.407038332584772, 0.7471403966175919, 0.43219718045477806]
Спектр: [0.18767422550023838, -0.005672845832269092, -4.1621621726549485]
Последняя точка: [12.377756424254523, 0.64553470348773, 0.4248180824565846]

---------
Индекс I0: 59; I0: -1.581030303030303
Индекс U0: 1; U0: 0.3
Начальные условия: [12.377756424254523, 0.64553470348773, 0.4248180824565846]
Спектр: [0.41894003032720173, -0.0008648814868591526, -4.377790322324739]
Последняя точка: [6.556823725336824, 0.7120831462594639, 0.4307281950241013]

---------
Индекс I0: 60; I0: -1.584151515151515
Индекс U0: 1; U0: 0.3
Начальные условия: [6.55682

Начальные условия: [10.85874072596154, 0.6484245471183873, 0.4302551210765811]
Спектр: [0.008225872813836398, -0.732106283581336, -5.013809797113162]
Последняя точка: [11.813799288450234, 0.6822670993935936, 0.44344683421673164]

---------
Индекс I0: 85; I0: -1.6621818181818182
Индекс U0: 1; U0: 0.3
Начальные условия: [11.813799288450234, 0.6822670993935936, 0.44344683421673164]
Спектр: [-0.00033064936722695807, -0.6866103148236904, -5.059143846497369]
Последняя точка: [10.434727437331732, 0.6596096763717684, 0.42230949933390194]

---------
Индекс I0: 86; I0: -1.6653030303030303
Индекс U0: 1; U0: 0.3
Начальные условия: [10.434727437331732, 0.6596096763717684, 0.42230949933390194]
Спектр: [0.0028464038654462055, -0.6325561597741136, -5.118147639085815]
Последняя точка: [10.580211043546722, 0.7398745967195879, 0.4313520934948293]

---------
Индекс I0: 87; I0: -1.6684242424242424
Индекс U0: 1; U0: 0.3
Начальные условия: [10.580211043546722, 0.7398745967195879, 0.4313520934948293]
Спектр: 


---------
Индекс I0: 12; I0: -1.4343333333333332
Индекс U0: 2; U0: 0.3017171717171717
Начальные условия: [17.221275741462904, 0.6024050456491558, 0.4289878957281778]
Спектр: [0.008159582056392356, -1.62580280768838, -3.5939379525158035]
Последняя точка: [10.315663410464634, 0.7816938639388264, 0.43378109340316395]

---------
Индекс I0: 13; I0: -1.4374545454545455
Индекс U0: 2; U0: 0.3017171717171717
Начальные условия: [10.315663410464634, 0.7816938639388264, 0.43378109340316395]
Спектр: [0.010247010678602024, -1.4905566319115806, -3.6958028533928964]
Последняя точка: [6.884048697901332, 0.815752297012518, 0.43157554681874977]

---------
Индекс I0: 14; I0: -1.4405757575757576
Индекс U0: 2; U0: 0.3017171717171717
Начальные условия: [6.884048697901332, 0.815752297012518, 0.43157554681874977]
Спектр: [0.004222023032265194, -1.3663745584430858, -3.783705283888323]
Последняя точка: [11.539862605783973, 0.7679001375229471, 0.43447953539097867]

---------
Индекс I0: 15; I0: -1.443696969696969

...

Последняя точка: [1.8703617548640814, 0.8931590533282111, 0.4307386014064607]

---------
Индекс I0: 71; I0: -1.6184848484848484
Индекс U0: 14; U0: 0.32232323232323234
Начальные условия: [1.8703617548640814, 0.8931590533282111, 0.4307386014064607]
Спектр: [0.8007650532592081, -0.0001963893912671636, -5.205273984018799]
Последняя точка: [7.662180381665954, 0.695440510916027, 0.41235662380795735]

---------
Индекс I0: 72; I0: -1.6216060606060605
Индекс U0: 14; U0: 0.32232323232323234
Начальные условия: [7.662180381665954, 0.695440510916027, 0.41235662380795735]
Спектр: [0.7179481640779274, -0.004996123371956222, -5.156220615623946]
Последняя точка: [3.439549224576098, 0.789993899968161, 0.4174808783718708]

---------
Индекс I0: 73; I0: -1.6247272727272728
Индекс U0: 14; U0: 0.32232323232323234
Начальные условия: [3.439549224576098, 0.789993899968161, 0.4174808783718708]
Спектр: [0.8132320210341604, -0.005122004325353272, -5.211403683457195]
Последняя точка: [7.952304404155403, 0.765665318

Последняя точка: [11.465515253973212, 0.6792533997987772, 0.43576598875098044]

---------
Индекс I0: 97; I0: -1.6996363636363636
Индекс U0: 14; U0: 0.32232323232323234
Начальные условия: [11.465515253973212, 0.6792533997987772, 0.43576598875098044]
Спектр: [-0.001043841302374831, -1.862198044849204, -4.6975781723002825]
Последняя точка: [13.792812985059415, 0.6368503063271318, 0.43044910199613723]

---------
Индекс I0: 98; I0: -1.7027575757575757
Индекс U0: 14; U0: 0.32232323232323234
Начальные условия: [13.792812985059415, 0.6368503063271318, 0.43044910199613723]
Спектр: [0.008193279873662841, -1.770780809750335, -4.86014570877757]
Последняя точка: [7.633352915404687, 0.8635151542848265, 0.4635760227681948]

---------
Индекс I0: 99; I0: -1.7058787878787878
Индекс U0: 14; U0: 0.32232323232323234
Начальные условия: [7.633352915404687, 0.8635151542848265, 0.4635760227681948]
Спектр: [0.01063321957813462, -1.7010114679893693, -4.9889069059153535]
Последняя точка: [7.382264108378948, 0.702


---------
Индекс I0: 24; I0: -1.4717878787878789
Индекс U0: 15; U0: 0.324040404040404
Начальные условия: [4.201752082622442, 0.834138564099431, 0.42225501179562064]
Спектр: [0.014007624760259248, -0.05813946028661652, -5.0747314382419]
Последняя точка: [4.6884729573868285, 0.7535385973052852, 0.41154370109756294]

---------
Индекс I0: 25; I0: -1.474909090909091
Индекс U0: 15; U0: 0.324040404040404
Начальные условия: [4.6884729573868285, 0.7535385973052852, 0.41154370109756294]
Спектр: [0.0024867748203945875, -0.1859987571738932, -4.934534952470795]
Последняя точка: [3.9359652300328984, 0.7847273497984598, 0.4125760379493245]

---------
Индекс I0: 26; I0: -1.478030303030303
Индекс U0: 15; U0: 0.324040404040404
Начальные условия: [3.9359652300328984, 0.7847273497984598, 0.4125760379493245]
Спектр: [0.01012706475491951, -0.34016612056638795, -4.781961443121896]
Последняя точка: [12.190413428466677, 0.7552819659022334, 0.4278874123433189]

---------
Индекс I0: 27; I0: -1.481151515151515
И


---------
Индекс I0: 50; I0: -1.5529393939393938
Индекс U0: 15; U0: 0.324040404040404
Начальные условия: [5.4908340933830475, 0.7372918253848296, 0.4096896522941368]
Спектр: [0.011508693828381579, -0.48100236367694055, -4.383392568035631]
Последняя точка: [3.7752867155732837, 0.7835442826173048, 0.41192644922687605]

---------
Индекс I0: 51; I0: -1.5560606060606061
Индекс U0: 15; U0: 0.324040404040404
Начальные условия: [3.7752867155732837, 0.7835442826173048, 0.41192644922687605]
Спектр: [0.008749247914415519, -1.1542536639744823, -3.6786254892533776]
Последняя точка: [14.243218016887733, 0.6769459116880922, 0.4222780529355639]

---------
Индекс I0: 52; I0: -1.5591818181818182
Индекс U0: 15; U0: 0.324040404040404
Начальные условия: [14.243218016887733, 0.6769459116880922, 0.4222780529355639]
Спектр: [0.004037494539189169, -1.069861641951694, -3.737980373321504]
Последняя точка: [4.0717167248717, 0.8669387678400918, 0.4325474677348054]

---------
Индекс I0: 53; I0: -1.5623030303030303

Excessive output truncated after 524352 bytes.

In [ ]:
lyapmin,lyapmax = minimum(Λs[:, :, 1]), maximum( Λs[:, :, 1])

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1], xlabel = L"I0",ylabel = L"U0", xlabelsize = 50, ylabelsize = 50,
            xticklabelsize = 35, yticklabelsize = 35,
            xgridvisible  = false, ygridvisible = false)

hm = heatmap!(ax1, I0range, U0range, Λs[:, :, 1], colormap = :seismic)
Colorbar(f[1, 2], hm)
f